In [31]:
ploting_region <- function(data,title) {
  ggplot(data, aes(x=Year, y=Expectancy, color=Region)) + 
  geom_point(size=4) + theme_ipsum() + labs(title=title)
}

ploting_country <- function(data,title) {
  ggplot(data, aes(x=Year, y=Expectancy, color=Country)) + 
  geom_point(size=4) + theme_ipsum() + labs(title=title)
}
library(tidyverse)
library(dplyr)
df =   read_csv("../input/human-life-expectancy-around-the-world/Human_life_Expectancy.csv")



#On voit que le data set n est pas bien representé alors on va le refactoriser en ajoutant nouvelle columne Year pour la valeur
#On rassemble les années dans une seul colonne en montrant la esperance correspondante par chaque année
long_DF <- df%>%gather(Year, Expectancy , `1990`:`2019`)
long_DF
glimpse(long_DF)

cat("Est ce que on a des valeurs null? ",is.null(long_DF))# on a pas de valuer null mais selon la description de data notre null est indique comme not available
cat(paste("\nCombien de valeurs Not Available contient data set "),sum(long_DF=='Not Available'))
cat(paste("\nLe pourcnetage de Not available sur notre data set "),sum(long_DF=='Not Available')/nrow(long_DF)*100)






#Certaines valeurs d'attente sont manquantes, indiquées comme "Not Available". 
#Et il existe des niveaux "national" et "subnational". 
#Nous travaillerons au niveau national. De plus, il n'y a pas de valeurs nulles. Nous supprimerons la colonne "region".
#La colonne "level" peut être supprimée après le filtrage des données au niveau national. 
#Nous avons besoin d'autres colonnes dans notre analyse. Nous nous occuperons des types de données après tout nettoyage.



#long_DF$Expectancy = as.numeric(long_DF$Expectancy) TODO BUT MAYBE AFTER
glimpse(long_DF)


data_all_national=long_DF %>% filter(long_DF$Level=='National')
data_all_national = select(data_all_national,-4)

summary(data_all_national)
glimpse(data_all_national)
#Dealing with Not Available

cat(paste("\nCombien de valeurs Not Available contient data set "),sum(data_all_national=='Not Available'))

In [32]:
#518 valeur ("Not Available") dans Expectancy sur niveau national

#On a quelques solution :soit on fait le drop ,soit on fait la regression pour predire les valeurs null,
#soit on remplit par la moyenne


#Drop

data_all_national_dropped=data_all_national %>% filter(data_all_national$Expectancy!='Not Available')
data_all_national_dropped$Expectancy = as.numeric(data_all_national_dropped$Expectancy)
data_all_national_dropped$Year = as.numeric(data_all_national_dropped$Year)
summary(data_all_national_dropped)


#On va reprednre le drop et on fait l'analyse global
#On commence par la  definition de correlation

library("ggpubr")
data_all_national_dropped_correlation = cor(data_all_national_dropped$Year,data_all_national_dropped$Expectancy)
print(data_all_national_dropped_correlation)
cat(paste("\nCa nous montre qu'il y a la correlation faible entre Year et Expectancy -> "),data_all_national_dropped_correlation)



In [33]:
#Regardons Scatter plot

#attach(data_all_national_dropped)
plot(data_all_national_dropped$Year, data_all_national_dropped$Expectancy, main="Scatterplot",
   xlab="Year ", ylab="Expectancy",col='blue')

#Entre 1990 et 2020, nous constatons que les valeurs minimales d'espérance de vie augmentent avec une certaine volatilité dans le monde, tandis que les valeurs maximales d'espérance de vie augmentent régulièrement. Si possible, nous vérifierons la raison de la volatilité des valeurs minimales d'espérance de vie à l'avenir.

#Nous voyons quelques valeurs aberrantes pour l'espérance de vie minimale entre 1990 et 1995, peut-être en raison de guerres, etc. Nous vérifierons quels pays ont ces valeurs, plus tard.

#(Ci-dessous ne sont que mes hypothèses)

#Une augmentation constante de l'espérance de vie minimale montre peut-être que les systèmes/technologies de santé progressent ou que l'accessibilité aux systèmes de santé augmente.

#Une lente augmentation de l'espérance de vie maximale montre qu'il n'existe pas encore de nouvelle technologie pour augmenter l'espérance de vie maximale et que les conditions de vie s'améliorent ou que le système de santé ne voit aucune nécessité de développement au-delà de 80 ans.

hist(data_all_national_dropped$Expectancy,xlab='Expectancy')

#Il a une asymétrie négative. La moyenne est plus petite que la médiane.

#Avec les statistiques ci-dessous, nous pouvons dire que vous verrez très probablement votre 70e-75e anniversaire.

In [34]:
summary(data_all_national_dropped)
cat("Notre ecart-type est ",sd(data_all_national_dropped$Expectancy))
#L'écart type est de 9,64. Étant donné que l'écart type est inférieur à la moyenne/3, il n'est pas élevé. 
#L'écart type avec une asymétrie négative indique qu'il s'écartera principalement vers des durées de vie plus longues.

#La valeur attendue minimale est (26,20) dans toutes les données. Et l'attente maximale est de 84,60.
#Regardons de quels pays il s'agit ?


In [35]:

country_with_lowest_level=data_all_national_dropped[data_all_national_dropped$Expectancy == 26.20, ] 
head(country_with_lowest_level)
country_with_highest_level=data_all_national_dropped[data_all_national_dropped$Expectancy == 84.60, ] 
head(country_with_highest_level)

#Maintenant, il est temps de rechercher les valeurs aberrantes.
#Les valeurs aberrantes commencent à 43,3 ( Q1 - (1,5 IQR) = (62,62 - (1,5 (75,70 - 62,62)) ).
#Pourtant, nous choisirons 43,00 pour voir plus de données près du seuil.

threshold_data=data_all_national_dropped %>% filter(data_all_national_dropped$Expectancy<=43)


print(as_tibble(threshold_data), n=100)



#Buthan a un peu de chaos vers 1990. Wikipédia - Bhoutan

#Le #igéria a un peu de chaos dans les années 1990. Wikipédia - Nigéria

#Il y a eu une guerre civile entre 1990 et 1994 au Rwanda. Wikipédia - Guerre civile rwandaise

#Il en est de même pour la Sierra Leone entre 1991-2002. Wikipédia - Guerre civile en Sierra Leone

#Nous n'avons pas encore assez de données pour Eswatini. Nous supposons que cela est dû à la pauvreté.

#Le Lesotho a connu la sécheresse en 2007. Wikipedia - Lesotho - Sécheresse

#Les guerres civiles expliquent majoritairement les décès entre 20 et 30 ans.



In [36]:
#Maintenant, il est temps d'examiner les statistiques et la distribution après avoir supprimé les valeurs aberrantes. Il n'y a pas de changements dramatiques
data_all_national_dropped_after_drop_outils=data_all_national_dropped %>% filter(data_all_national_dropped$Expectancy>43)

summary(data_all_national_dropped_after_drop_outils)

hist(data_all_national_dropped_after_drop_outils$Expectancy)



In [37]:
#Regardons 80>
highest=data_all_national_dropped %>% filter(data_all_national_dropped$Expectancy>=80)
head(highest)


library(ggplot2)
library(hrbrthemes)

options(repr.plot.width=15, repr.plot.height=8)
ggplot(highest, aes(x=Year, y=Expectancy, color=Country)) + 
    geom_point(size=4) +
    theme_ipsum() + labs(title="Highest expectancy >80")

#La plupart du temps, les pays développés ont une espérance de vie maximale de plus de 80 ans. Le Japon est en tête de la course.
#Les shinigamis sont paresseux dans leur continent.

#Maintenant, nous examinerons une espérance de vie inférieure à 50 ans et ignorerons les valeurs aberrantes puisque nous en connaissons la raison.

medium=data_all_national_dropped %>% filter(data_all_national_dropped$Expectancy>=35 & data_all_national_dropped$Expectancy<50 )

summary(medium)

options(repr.plot.width=15, repr.plot.height=8)

ploting_country(medium,"Medium expectancy >=35 and <80")

In [38]:
#L'intrigue complète de l'espérance à travers le temps
plot(data_all_national_dropped$Year, data_all_national_dropped$Expectancy, pch = 16, cex = 1.3, col = "red", main = "Expectancy throught the time", xlab = "Year", ylab = "Expectancy ")

# Les valeurs augmentent d'année en année à l'échelle mondiale. Surtout le pays qui avait une espérance de faible niveau a obtenu un bon coup de pouce
#Le map

In [111]:
library("ggplot2")
theme_set(theme_bw())
library("sf")
library("rnaturalearth")
library("rnaturalearthdata")

world <- ne_countries(scale = "medium", returnclass = "sf")
class(world)

#Regardons la carte de 2019 par rapport a l'expectance de vie
data_2019=data_all_national_dropped %>% filter(data_all_national_dropped$Year==2019)


print(world[1,"name"]$geometry)
colonne_zero=c()
for (i in 1:241){
    colonne_zero=c(colonne_zero,0)
}
world['Expectancy']=colonne_zero
len_world=nrow(world)
len_data=nrow(data_2019)
new_data=c()
for (index in 1:len_world){
    for(index2 in 1:len_data){
        if (world[index,]$name == data_2019[ index2,]$Country){
            world[index,]$Expectancy=data_2019[ index2,]$Expectancy
        }
    }
  
}



In [112]:
#La carte enfin
ggplot(data = world) +
    geom_sf(aes(fill = Expectancy)) +
    scale_fill_viridis_c(option = "plasma", trans = "sqrt")


In [118]:
#Russie n'est pas enregistré dans la base de donnees en 2019,alors c'est pour ca ca nous montre 0
wolrd_russia=world %>% filter(world$name=='Russia')
head(wolrd_russia)
data_russia=data_2019 %>% filter(data_2019$Country=='Russia')
head(data_russia)

In [39]:
#Maintenat on peut prendle le pays exact et voir differentes regions durant le temps
#On va prendre la France pq pas?
data_FR=long_DF %>% filter(long_DF$Country=='France')
head(data_FR)
data_FR=data_FR %>% filter(data_FR$Region !='Total')
#Transforme sur les valeurs numeric
data_FR$Expectancy = as.numeric(data_FR$Expectancy) 
data_FR$Year = as.numeric(data_FR$Year) 
summary(data_FR)

options(repr.plot.width=25, repr.plot.height=8)

ploting_region(data_FR,"France during the time")

In [40]:
#Alors ,on voit que tous les valeurs grandissent durant le temps sauf une.La region Mayotte
#On prennons cette region pour etudier

data_Mayotte=data_FR %>% filter(data_FR$Region =='Mayotte')
summary(data_Mayotte)
#On voit que la valeur min est max presque identique
cat("L'ecart type de Mayotte ",sd(data_Mayotte$Expectancy))#L'escart type est tre faible.

ploting_region(data_Mayotte,"Mayotte during the time")
#History Wikipedia
#Le mouvement social de 2018 à Mayotte, aussi appelé grève générale de 2018 à Mayotte ou opération « Île morte », est un mouvement social qui se produit à Mayotte dès le début de l'année 2018. 
#Il fait suite au mouvement populaire de mobilisation contre un accord franco-comorien de septembre 2017 vers la libre circulation des personnes entre l'Union des Comores et Mayotte

In [41]:
#Par contre on peut voir la grosse expectance au debut sur une region
#On va decouvrir cette region

data_reg=data_FR %>% filter(data_FR$Year == 1990)

data_reg_sort <- data_reg[order(-data_reg$Expectancy),]
data_reg_sort
cat("Cette Region est Guadeloupe\n")
data_Guade=data_FR %>% filter(data_FR$Region == 'Guadeloupe')
ploting_region(data_Guade,'Guadeloupe during the time')
#On voit que a partir 2006 jusqu'a 2014 il y a la valeur tres faible sur l'Expectance
#History